In [1]:
import geopandas as gpd
import pandas as pd
from glob import glob

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.11.0-CAPI-1.17.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
kdprov=['51']
nmfiledegurba="degurba_Bali_00.gpkg"

In [3]:
data_degurba_grod=gpd.read_file('data/output/2_grid_degurba/'+nmfiledegurba)

In [4]:
data_pop=gpd.GeoDataFrame()
for i in kdprov:
    gl_pop=glob("data/output/1_population_density/grid_pop_"+i+"*_100m.gpkg")
    for j in gl_pop:
        y_temp=gpd.read_file(j)
        y_temp['geometry']=y_temp.centroid
        y_temp=y_temp[['j_pend','geometry','p_area']]
        data_pop=y_temp.append(data_pop)

/tmp/ipykernel_4360/399264284.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_temp.append(data_pop)
/tmp/ipykernel_4360/399264284.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_temp.append(data_pop)
/tmp/ipykernel_4360/399264284.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_temp.append(data_pop)
/tmp/ipykernel_4360/399264284.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_temp.append(data_pop)
/tmp/ipykernel_4360/399264284.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_te

In [5]:
data_desa_raster=gpd.GeoDataFrame()
for i in kdprov:
    temp=gpd.read_file('data/output/0_raster_data_desa/'+'desa_pend_rasterize_'+i+'.gpkg')
    data_desa_raster=temp.append(data_desa_raster)

/tmp/ipykernel_4360/2837293180.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_desa_raster=temp.append(data_desa_raster)


In [6]:
pop_w_grid=data_pop.sjoin_nearest(data_degurba_grod)[['j_pend','classification','geometry']]

In [7]:
data_desa=data_desa_raster.sjoin(pop_w_grid)[['j_pend','classification','iddesa','geometry']]

In [8]:
data_desa

,j_pend,classification,iddesa,geometry
0,0.0,RURAL,5101010001,"MULTIPOLYGON (((11397700.000 -1007700.000, 113..."
0,0.0,RURAL,5101010001,"MULTIPOLYGON (((11397700.000 -1007700.000, 113..."
0,0.0,RURAL,5101010001,"MULTIPOLYGON (((11397700.000 -1007700.000, 113..."
0,0.0,RURAL,5101010001,"MULTIPOLYGON (((11397700.000 -1007700.000, 113..."
0,0.0,RURAL,5101010001,"MULTIPOLYGON (((11397700.000 -1007700.000, 113..."
...,...,...,...,...
715,0.0,CENTRE,5171031011,"POLYGON ((11468200.000 -1060800.000, 11468300...."
715,0.0,CENTRE,5171031011,"POLYGON ((11468200.000 -1060800.000, 11468300...."
715,0.0,CENTRE,5171031011,"POLYGON ((11468200.000 -1060800.000, 11468300...."
715,0.0,CENTRE,5171031011,"POLYGON ((11468200.000 -1060800.000, 11468300...."


In [9]:
tab_data=pd.pivot_table(data_desa,index='iddesa',columns='classification',
                        values='j_pend',aggfunc='sum').fillna(0).reset_index()

In [10]:
l_col=['CENTRE','CLUSTER','RURAL']
for i in l_col:
    if i not in tab_data.columns:
        tab_data[i]=0
tab_data

classification,iddesa,CENTRE,CLUSTER,RURAL
0,5101010001,0.0,12757.641694,308.358306
1,5101010002,0.0,2093.552622,3365.447378
2,5101010003,0.0,3701.690436,1403.309564
3,5101010004,0.0,6245.129550,1271.870450
4,5101010005,0.0,7286.623370,1160.376630
...,...,...,...,...
711,5171031007,11690.0,0.000000,0.000000
712,5171031008,25498.0,0.000000,0.000000
713,5171031009,17754.0,0.000000,0.000000
714,5171031010,37716.0,0.000000,0.000000


In [11]:
def define_LAU(y):
    ['CENTRE','CLUSTER','RURAL']
    total_pop=y.CENTRE+y.CLUSTER+y.RURAL
    if total_pop==0:
        return 'RURALS'
    elif (y.CENTRE/total_pop)>=0.5:
        return 'CITIES'
    elif (((y.CENTRE/total_pop)<0.5)&((y.RURAL/total_pop)<0.5)):
        return 'TOWNS'
    else:
        return 'RURALS'

In [12]:
tab_data['LAU']=tab_data.apply(lambda y:define_LAU(y),axis=1)

In [13]:
peta_desa=gpd.read_file(r"data/input/Master_Data/DESA_20201_BYINDO.gpkg").query('kdprov in @kdprov')

In [14]:
peta_desa_LAU=peta_desa.merge(tab_data[['iddesa','LAU']],how='left').fillna('RURALS')

In [15]:
for i in kdprov:
    peta_desa_LAU.query("kdprov==@i").to_file(
        'data/output/3_small_spatial_unit_degurba/small_spatial_unit_'+i+'.gpkg',driver='GPKG')

In [16]:
from dbfread import DBF
mfd_=pd.DataFrame(iter(DBF('data/input/Master_Data/mfd_2020_1.dbf')))

In [17]:
mfd_['iddesa']=mfd_.PROP+mfd_.KAB+mfd_.KEC+mfd_.DESA

In [18]:
for i in kdprov:
    peta_desa_LAU.merge(mfd_[['iddesa','DRH']]).query("kdprov==@i").to_file(
        'data/output/3_small_spatial_unit_degurba/small_spatial_unit_'+i+'.gpkg',driver='GPKG')

In [19]:
peta_desa_LAU.merge(mfd_[['iddesa','DRH']]).pivot_table(index='DRH',columns='LAU',values='iddesa',aggfunc='count').fillna(0)

LAU,CITIES,RURALS,TOWNS
DRH,,,
1,160,13,96
2,37,88,322
